## Neural DB distributed training

In this notebook, you will be able to train ThirdAI's Neural DB on 10k Amazon datapoints in a distributed fashion.

In [1]:
!pip3 install thirdai --upgrade
!pip3 install thirdai[neural_db]
!pip3 install ray
!pip3 install torch

from thirdai import neural_db
import thirdai.distributed_bolt as dist

/home/mritunjay/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-21 08:53:41,518	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-21 08:53:44,455	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-21 08:53:44,655	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
## Activate your license here.

from thirdai import licensing
licensing.activate("your-license-key-here")

In [2]:
## Download the data file

import os

filename = "amazon_10k.csv"
os.system(f"wget -O {filename} 'https://www.dropbox.com/scl/fi/97utx7ukp0rb37f8d98ia/amazon_10k.csv?rlkey=aq8yq42o54tcj62u9q3op0m80&dl=0'")

--2023-08-21 08:53:50--  https://www.dropbox.com/scl/fi/97utx7ukp0rb37f8d98ia/amazon_10k.csv?rlkey=aq8yq42o54tcj62u9q3op0m80&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:601b:18::a27d:812, 162.125.8.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:601b:18::a27d:812|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/e/scl/fi/97utx7ukp0rb37f8d98ia/amazon_10k.csv?dl=0&rlkey=aq8yq42o54tcj62u9q3op0m80 [following]
--2023-08-21 08:53:50--  https://www.dropbox.com/e/scl/fi/97utx7ukp0rb37f8d98ia/amazon_10k.csv?dl=0&rlkey=aq8yq42o54tcj62u9q3op0m80
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5dd8cd2b16134e7c3a448659a0.dl.dropboxusercontent.com/cd/0/get/CCOUzKUIKqCZlGpLPb17N85kjx1XyWRhMJxmuk0n9voK0peJeVdT7FpoKJcYYVedcQumSpj4nDfIoLKnyrNyAd_jZxh8PKiGmjjUbRSZ1dZBYYFj6WTyRTC9aYmhFC7f2y2ydJ3nEWNQG80-tyF1qpUs/file# [following]
--2023-08-21 08:53:5

0

In [3]:
ndb = neural_db.NeuralDB("my_user") # you can use any username, in the future, this username will let you push models to the model bazaar

doc = neural_db.CSV(
    filename,
    id_column="id",
    strong_columns=["TITLE", "BULLET_POINTS"],
    weak_columns=["DESCRIPTION"],
    reference_columns=["TITLE"],
)

In [4]:
## Insert the document to be indexed

ndb.insert(sources=[doc], train=False)

['64d270ee5d54cbeba6fde01e97096840fb540263']

In [5]:
## We initialize a two-node ray cluster

import ray
from ray.air import ScalingConfig, session

cpus_per_node = (dist.get_num_cpus() - 1) // 2

ray.init(ignore_reinit_error=True, runtime_env={"env_vars": {"OMP_NUM_THREADS": f"{cpus_per_node}"}})
scaling_config = ScalingConfig(
    num_workers=2,
    use_gpu=False,
    trainer_resources={"CPU": 1},
    resources_per_worker={"CPU": cpus_per_node},
    placement_strategy="PACK",
)

2023-08-21 08:54:14,278	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [7]:
## Runs distributed training on the document

ndb.pretrain_distributed(documents=[doc], scaling_config=scaling_config, epochs=5)

(BoltTrainer pid=2507942) Starting distributed worker processes: ['2508027 (192.168.1.6)', '2508028 (192.168.1.6)']
(RayTrainWorker pid=2508027) Setting up process group for: env:// [rank=0, world_size=2]
(SplitCoordinator pid=2508195) Auto configuring locality_with_output=['2cb6a39f8b218a12d29aadfd29f53ebd2d422140f3f0c1b4141b5d31', '2cb6a39f8b218a12d29aadfd29f53ebd2d422140f3f0c1b4141b5d31']
(SplitCoordinator pid=2508195) Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV->SplitBlocks(96)] -> OutputSplitter[split(2, equal=True)]
(SplitCoordinator pid=2508195) Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=2000000000.0), locality_with_output=['2cb6a39f8b218a12d29aadfd29f53ebd2d422140f3f0c1b4141b5d31', '2cb6a39f8b218a12d29aadfd29f53ebd2d422140f3f0c1b4141b5d31'], preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(SplitCoordinator pid=2508195) Tip: For detailed progress reporting, run `

(RayTrainWorker pid=2508028) loading data | source 'ray-dataset-sources'
(RayTrainWorker pid=2508028) loaded data | source 'ray-dataset-sources' | vectors 33178 | batches 17 | time 0.223s | complete
(RayTrainWorker pid=2508028) 
train: [                                                  ] 0%          
train: [===                                               ] 5%
(RayTrainWorker pid=2508027) loading data | source 'ray-dataset-sources'
(RayTrainWorker pid=2508027) loaded data | source 'ray-dataset-sources' | vectors 33948 | batches 17 | time 0.237s | complete
(RayTrainWorker pid=2508027) 
train: [                                                  ] 0%          
train: [======                                            ] 11%
train: [=========                                         ] 17%
train: [===                                               ] 5%
train: [============                                      ] 23%
train: [======                                            ] 11%
train: [======

2023-08-21 09:00:25,451	INFO tune.py:1148 -- Total run time: 136.43 seconds (136.21 seconds for the tuning loop).


### Let's ask!

In [8]:
search_results = ndb.search(query="Modern Contemporary Three Level Rotating Coffee Table for Living Room", top_k=3)

In [9]:
for result in search_results:
    print(result.text)
    #print(result.source)
    print('************')

TITLE: Zivite Armor Shockproof Soft TPU and Hard PC Back Cover Case with Magnetic Ring Holder for Oppo A3s - Armor Grey
/home/mritunjay/Demos/neural_db_examples/amazon_10k.csv
************
TITLE: Atlanta Post Caps 4x4 Post Cap | Brown England Pyramid Style Square Top for Outdoor Fences, Mailboxes & Decks
/home/mritunjay/Demos/neural_db_examples/amazon_10k.csv
************
TITLE: Ignition Coil Tester, Coil On Plug Tester, Battery Powered, Durable Accurate And Reliable Factory Maintenance Worker for Home Laboratory
/home/mritunjay/Demos/neural_db_examples/amazon_10k.csv
************


In [ ]:
## Terminate the ray cluster

ray.shutdown()